In [1]:
# Importing the required libraries#
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [2]:
cnn_model = keras.models.load_model(r"C:\Users\mahen\Final_CNN_Model.h5")

In [3]:
# initializing the variables for the roi(region of intrest), bg(background), accum_weight
bg = None
accum_weight = 0.5

ROI_top = 100
ROI_bottom = 500
ROI_right = 150
ROI_left = 550

In [4]:
# function to detect background without hand
def cal_accum_avg(frame, accum_weight):
    global bg

    if bg is None:
        bg = frame.copy().astype("float")
        return None

    cv2.accumulateWeighted(frame, bg, accum_weight)

In [5]:
# function to detect the hand and contours and return the thresholded image and max contours
def segment_hand(frame, threshold = 15):
    global bg

    diff = cv2.absdiff(bg.astype("uint8"), frame)

    _, thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)

    # Grab the external contours for the image
    contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) == 0:
        return None
    else:

        hand_segment_max_cont = max(contours, key=cv2.contourArea)

        return (thresholded, hand_segment_max_cont)

In [7]:
# initializing the object for video capture
logi = cv2.VideoCapture(0, cv2.CAP_DSHOW)

num_frames = 0

while True:
    ret, frame = logi.read()
    frame = cv2.resize(frame, (960, 720))
     # filpping the frame to prevent inverted image of captured frame...
    frame = cv2.flip(frame, 1)
    frame_copy = frame.copy()

    roi = frame[ROI_top:ROI_bottom, ROI_right:ROI_left]

    gray_frame = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray_frame = cv2.GaussianBlur(gray_frame, (9, 9), 0)

    if num_frames < 60:
        cal_accum_avg(gray_frame, accum_weight)
        if num_frames <= 59:
            cv2.putText(frame_copy, "Please wait until the background is detected.", (80, 570), cv2.FONT_HERSHEY_SIMPLEX, 0.7,
                        (0, 0, 255), 2)
    elif num_frames < 100: 
        cv2.putText(frame_copy, "Background detection successful.Place your hand in the roi for detection.", 
                    (80, 570), cv2.FONT_HERSHEY_SIMPLEX, 0.7,(0, 0, 255), 2)
    else:
        # segmenting the hand region
        hand = segment_hand(gray_frame)
        

        # Checking if we are able to detect the hand...
        if hand is not None:
            
            thresholded, hand_segment = hand

            # Drawing contours around hand segment
            cv2.drawContours(frame_copy, [hand_segment + (ROI_right, ROI_top)], -1, (255, 0, 0),1)
            
            cv2.imshow("Thesholded Hand Image", thresholded)
            
            thresholded = cv2.resize(thresholded, (64, 64))
            
            thresholded = cv2.cvtColor(thresholded, cv2.COLOR_GRAY2RGB)
            
            thresholded = np.reshape(thresholded, (1,thresholded.shape[0],thresholded.shape[1],3))
            
            pred = cnn_model.predict(thresholded)
            
            word_dict = {0:'Zero',1:'One',2:'Two',3:'Three',4:'Four',5:'Five',6:'Six',7:'Seven',8:'Eight',9:'Nine'}
            
            cv2.putText(frame_copy, word_dict[np.argmax(pred)], (170, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
    # Draw ROI on frame_copy
    cv2.rectangle(frame_copy, (ROI_left, ROI_top), (ROI_right, ROI_bottom), (255,128,0), 3)

    # counting the number of frames for tracking
    num_frames += 1

    # Display the frame with segmented hand
    cv2.imshow("Sign Detection", frame_copy)


    # Close windows with Esc
    k = cv2.waitKey(1) & 0xFF

    if k == 27:
        break
        
# Release the camera and destroy all the windows
logi.release()
cv2.destroyAllWindows()
